In [ ]:
import argparse
import os

if hasattr(__builtins__,'__IPYTHON__'):
    
    sysName='DY'
    sysfiles='/eos/cms/store/group/phys_egamma/akapoor/ChargeMisID/newsamples2/*.root'
    saveRootFile=False
    outfolder='Output'
    outsuffix='1'
    branchlist="(ChargeFlip_Category|MyElectron_pt)"
else:
    parser = argparse.ArgumentParser(description='Options for the analyzer')
    parser.add_argument('--name', metavar='n', action='store', type=str,help='short name of the physics process/Data sample', required=True)
    parser.add_argument('--file', metavar='f', action='store', type=str,help='location of files',required=True)
    parser.add_argument('--saveroot',action='store_true', help='save root file',default=False)
    parser.add_argument('--outfolder', metavar='of', action='store', type=str,help='location of output',default='Output')
    parser.add_argument('--outsuffix', metavar='os', action='store', type=str,help='suffix to recognize job',default='Output')
    parser.add_argument('--branchlist', metavar='bl', action='store', type=str,help='branches to store in skim',default="(MyElectron_eta|MyElectron_pt)")
    args = parser.parse_args()
    sysName=str(args.name)
    sysfiles=str(args.file)
    saveRootFile=args.saveroot
    outfolder=str(args.outfolder)
    outsuffix=str(args.outsuffix)
    branchlist=str(args.branchlist)

os.system("mkdir "+outfolder)


import ROOT
ROOT.ROOT.EnableImplicitMT()
ROOT.gInterpreter.AddIncludePath("./include/");
ROOT.gInterpreter.ProcessLine('#include "Helper.h"')
from include.PyHelper import *

ROOT.gStyle.SetOptStat(0)

Ana= MyAnalyzer()

Ana.process(sysName,{'Files':sysfiles,'Type':'Bkg','Tree':'Events','Xsec':1})
Ana.prepare()

###########################################################################################################################

In [ ]:
hMeeSS = Ana.PD[sysName]['RDF'].Filter("(elq1+elq2)!=0", "Two SS").Histo1D(("ChargeFlip_Category_SS", "ChargeFlip_Category;ChargeFlip_Category;Events", 23, 0.5, 23.5), "ChargeFlip_Category")
hMee = Ana.PD[sysName]['RDF'].Histo1D(("ChargeFlip_Category_all", "ChargeFlip_Category;ChargeFlip_Category;Events", 23, 0.5, 23.5), "ChargeFlip_Category")

savehist([hMee.Clone(),hMeeSS.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],norm=False,outfolder=outfolder)
savehistonlyratio([hMeeSS.Clone(),hMee.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],outfolder=outfolder)


###pt
hpt1SS = Ana.PD[sysName]['RDF'].Filter("(elq1+elq2)!=0", "Two SS").Histo1D(("pt1_SS", "pt1;pt1;Events", 5, 10, 100), "elpt1")
hpt1 = Ana.PD[sysName]['RDF'].Histo1D(("pt1_all", "pt1;pt1;Events", 5, 10, 100), "elpt1")

savehist([hpt1.Clone(),hpt1SS.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],norm=False,outfolder=outfolder)
savehistonlyratio([hpt1SS.Clone(),hpt1.Clone()],[ROOT.kRed,ROOT.kBlue,ROOT.kGreen],outfolder=outfolder)


DF=Ana.PD[sysName]['RDF']

if saveRootFile:
    #std::vector<string> save_cols ={"Electron_pt"}
    DF.Snapshot("Events",outfolder+"/output_"+outsuffix+".root",branchlist)

from uncertainties import ufloat
import numpy as np
SS=DF.Filter("(elq1+elq2)!=0", "Two SS").Count().GetValue()
All=DF.Count().GetValue()
SS=ufloat(SS,np.sqrt(SS))
All=ufloat(All,np.sqrt(All))

GlobalRate=SS/All

print(f"The global charge flip rate is {GlobalRate*100}%")